# Preprocessing brain MRI of patients with meningioma

## Before using MONSTR

### Setting PATH variable
**The environment PATH MUST be set in the main terminal, not otherwise like in a sub-terminal in the Jupyterlab**

#### ANTs
export ANTSPATH=/opt/ANTs/bin/

export PATH=${ANTSPATH}:$PATH

#### FSL
FSLDIR=/usr/local/fsl
. ${FSLDIR}/etc/fslconf/fsl.sh

PATH=${FSLDIR}/bin:${PATH}

export FSLDIR PATH


### Update
6/22 \
Code that applies brain mask added.

6/23 \
New variable T1C_label added\
(Sometimes you might want to do co-registration to sequence other than T1C, but this time we assume that you are  registering other sequences to T1C.)\

7/6 \
New variable mask_label added

7/7 \
Add try-except when finding T1Cpath and T2path using glob since T1C and T2 file might not exist

In [1]:
import os
import shutil
import nibabel as nib
import numpy as np
from glob import glob

In [2]:
def make_dirs(path_lst):
    for path in path_lst:
        if not os.path.exists(path):
            os.makedirs(path)
        else:
            pass

# make outputdir and return output path (including filename) by combining outDir, pt, filename
def get_outpath(outDir, pt, filename):
    outPath = os.path.join(outDir, pt, filename)

    make_dirs([os.path.join(outDir, pt)])

    if os.path.exists(outPath):
        print('{} already exists'.format(filename))
        return None
    else:
        return outPath

# find sequence from filename
def find_seq(filename):
    seq_lst = ["T1_GD", "T1GD", "T1CE", "T1C", "T1", "T2", "FLAIR", "F", "FLAIRGD", "ADC", "BB", "WB"]
    seq = None
    for i in seq_lst:
        if i in filename.upper():
            seq = i
            return seq
    if seq is None:
        print("No matching sequence")
        return None

In [6]:
sourceDir = '/media/sc-mlsg/PYW98854602/20210104_Schizophrenia/HCs/'
targetDir = '/media/sc-mlsg/PYW98854602/20210104_Schizophrenia_preprocessed/HCs/'
maskDir = None
outputPath = '/home/scmlsg/MONSTR/output'

resampleDir_mask = os.path.join(targetDir, "00_Tumor_masks_resampled")
resampleDir = os.path.join(targetDir, "01_Resampled")
correctionDir = os.path.join(targetDir, "02_Corrected")
registerDir = os.path.join(targetDir, '03_Registered_corrected')
brainmaskDir = os.path.join(targetDir, 'BrainMask_corrected')
strippedDir = os.path.join(targetDir, '04_Stripped_wo_Normalized')
znormedDir = os.path.join(targetDir, '05_Stripped_znormalized')

make_dirs([targetDir, resampleDir, correctionDir, registerDir, brainmaskDir, strippedDir, znormedDir])
make_dirs([resampleDir_mask])

T1C_label = 'T1' # 'BB','T1C','T1GD' etc.
mask_label = 'mask' # '_mask', '-label' etc.

In [7]:
# check directory structure
for (path, dirs, files) in os.walk(sourceDir):
    print(path, dirs, files)

/media/sc-mlsg/PYW98854602/20210104_Schizophrenia/HCs/ ['031', '068', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '012', '013', '014', '015', '017', '018', '019', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '032', '033', '034', '035', '036', '037', '038', '039', '040', '042', '043', '044', '046', '047', '048', '049', '051', '062', '063', '064', '065', '066', '067', '069', '070', '071', '072', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '085', '086', '087', '088', '089', '090', '092', '093', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112'] []
/media/sc-mlsg/PYW98854602/20210104_Schizophrenia/HCs/031 ['HC31_recon'] ['1_HC_0031_dti_ADC.nii.gz', '1_HC_0031_dti_FA_FA.nii.gz', 'T1.nii', 'T1.nii.gz']
/media/sc-mlsg/PYW98854602/20210104_Schizophrenia/HCs/031/HC31_recon ['label', 'mri', 'scripts', 'stats', 'surf', 'tmp', 'touch', 'trash'] []


/media/sc-mlsg/PYW98854602/20210104_Schizophrenia/HCs/069/HC69_recon/stats [] ['aseg.stats', 'lh.aparc.a2009s.stats', 'lh.aparc.DKTatlas.stats', 'lh.aparc.pial.stats', 'lh.aparc.stats', 'lh.BA_exvivo.stats', 'lh.BA_exvivo.thresh.stats', 'lh.curv.stats', 'lh.w-g.pct.stats', 'rh.aparc.a2009s.stats', 'rh.aparc.DKTatlas.stats', 'rh.aparc.pial.stats', 'rh.aparc.stats', 'rh.BA_exvivo.stats', 'rh.BA_exvivo.thresh.stats', 'rh.curv.stats', 'rh.w-g.pct.stats', 'wmparc.stats']
/media/sc-mlsg/PYW98854602/20210104_Schizophrenia/HCs/069/HC69_recon/surf [] ['lh.area', 'lh.area.mid', 'lh.area.pial', 'lh.avg_curv', 'lh.curv', 'lh.curv.pial', 'lh.defect_borders', 'lh.defect_chull', 'lh.defect_labels', 'lh.inflated', 'lh.inflated.H', 'lh.inflated.K', 'lh.inflated.nofix', 'lh.jacobian_white', 'lh.orig', 'lh.orig.nofix', 'lh.pial', 'lh.qsphere.nofix', 'lh.smoothwm', 'lh.smoothwm.BE.crv', 'lh.smoothwm.C.crv', 'lh.smoothwm.H.crv', 'lh.smoothwm.K.crv', 'lh.smoothwm.K1.crv', 'lh.smoothwm.K2.crv', 'lh.smoothwm.

In [14]:
sourceDir = glob('/media/sc-mlsg/PYW98854602/20210104_Schizophrenia/HCs/**/T1.nii.gz')

# 1) resampling
total = 0
for filePath in sourceDir:
    total += 1
    path = os.path.dirname(filePath)
    file = os.path.basename(filePath)
    print("Resampling: #{}, {}".format(total, file))
    pt = path.split(os.path.sep)[-1]
    if mask_label in file:
        pt = file.split("_")[0]
        resample_outPath = get_outpath(resampleDir_mask, pt, file)
    else:
        resample_outPath = get_outpath(resampleDir, pt, file)

    if resample_outPath is None:
        continue
    !ResampleImage 3 {filePath} {resample_outPath} 1x1x1

Resampling: #1, T1.nii.gz
T1.nii.gz already exists
Resampling: #2, T1.nii.gz
T1.nii.gz already exists
Resampling: #3, T1.nii.gz
Resampling: #4, T1.nii.gz
Resampling: #5, T1.nii.gz
Resampling: #6, T1.nii.gz
Resampling: #7, T1.nii.gz
Resampling: #8, T1.nii.gz
Resampling: #9, T1.nii.gz
Resampling: #10, T1.nii.gz
Resampling: #11, T1.nii.gz
Resampling: #12, T1.nii.gz
Resampling: #13, T1.nii.gz
Resampling: #14, T1.nii.gz
Resampling: #15, T1.nii.gz
Resampling: #16, T1.nii.gz
Resampling: #17, T1.nii.gz
Resampling: #18, T1.nii.gz
Resampling: #19, T1.nii.gz
Resampling: #20, T1.nii.gz
Resampling: #21, T1.nii.gz
Resampling: #22, T1.nii.gz
Resampling: #23, T1.nii.gz
Resampling: #24, T1.nii.gz
Resampling: #25, T1.nii.gz
Resampling: #26, T1.nii.gz
Resampling: #27, T1.nii.gz
Resampling: #28, T1.nii.gz
Resampling: #29, T1.nii.gz
Resampling: #30, T1.nii.gz
Resampling: #31, T1.nii.gz
Resampling: #32, T1.nii.gz
Resampling: #33, T1.nii.gz
Resampling: #34, T1.nii.gz
Resampling: #35, T1.nii.gz
Resampling: #3

In [40]:
# 1) resampling
total = 0
for (path, dirs, files) in os.walk(sourceDir):
    if dirs:
        for file in files:
            total += 1
            print("Resampling: #{}, {}".format(total, file))
            filePath = os.path.join(path, file)
            pt = path.split(os.path.sep)[-1]
            if mask_label in file:
                pt = file.split("_")[0]
                resample_outPath = get_outpath(resampleDir_mask, pt, file)
            else:
                resample_outPath = get_outpath(resampleDir, pt, file)

            if resample_outPath is None:
                continue
            !ResampleImage 3 {filePath} {resample_outPath} 1x1x1

Resampling: #1, 2034571_ADC.nii.gz
Resampling: #2, 2034571_FLAIR.nii.gz
Resampling: #3, 2034571_mask.nii
Resampling: #4, 2034571_T1GD.nii.gz
Resampling: #5, 2803895_ADC.nii.gz
Resampling: #6, 2803895_FLAIR.nii.gz
Resampling: #7, 2803895_mask.nii
Resampling: #8, 2803895_T1GD.nii.gz
Resampling: #9, 2826490_ADC.nii.gz
Resampling: #10, 2826490_FLAIR.nii.gz
Resampling: #11, 2826490_mask.nii
Resampling: #12, 2826490_T1GD.nii.gz
Resampling: #13, 2917442_ADC.nii.gz
Resampling: #14, 2917442_FLAIR.nii.gz
Resampling: #15, 2917442_mask.nii
Resampling: #16, 2917442_T1GD.nii.gz
Resampling: #17, 3387356_ADC.nii.gz
Resampling: #18, 3387356_FLAIR.nii.gz
Resampling: #19, 3387356_mask.nii
Resampling: #20, 3387356_T1GD.nii.gz
Resampling: #21, 3441152_ADC.nii.gz
Resampling: #22, 3441152_FLAIR.nii.gz
Resampling: #23, 3441152_mask.nii
Resampling: #24, 3441152_T1GD.nii.gz
Resampling: #25, 3579055_ADC.nii.gz
Resampling: #26, 3579055_FLAIR.nii.gz
Resampling: #27, 3579055_mask.nii
Resampling: #28, 3579055_T1GD.n

Resampling: #224, 7955082_T1GD.nii.gz
Resampling: #225, 7968300_ADC.nii.gz
Resampling: #226, 7968300_FLAIR.nii.gz
Resampling: #227, 7968300_mask.nii
Resampling: #228, 7968300_T1GD.nii.gz
Resampling: #229, 7971571_ADC.nii.gz
Resampling: #230, 7971571_FLAIR.nii.gz
Resampling: #231, 7971571_mask.nii
Resampling: #232, 7971571_T1GD.nii.gz
Resampling: #233, 7973273_ADC.nii.gz
Resampling: #234, 7973273_FLAIR.nii.gz
Resampling: #235, 7973273_mask.nii
Resampling: #236, 7973273_T1GD.nii.gz
Resampling: #237, 8002014_ADC.nii.gz
Resampling: #238, 8002014_FLAIR.nii.gz
Resampling: #239, 8002014_mask.nii
Resampling: #240, 8002014_T1GD.nii.gz
Resampling: #241, 8011880_ADC.nii.gz
Resampling: #242, 8011880_FLAIR.nii.gz
Resampling: #243, 8011880_mask.nii
Resampling: #244, 8011880_T1GD.nii.gz
Resampling: #245, 8018918_ADC.nii.gz
Resampling: #246, 8018918_FLAIR.nii.gz
Resampling: #247, 8018918_mask.nii
Resampling: #248, 8018918_T1GD.nii.gz
Resampling: #249, 8037487_ADC.nii.gz
Resampling: #250, 8037487_FLAIR

Resampling: #444, 8633978_T1GD.nii.gz
Resampling: #445, 8640861_ADC.nii.gz
Resampling: #446, 8640861_FLAIR.nii.gz
Resampling: #447, 8640861_mask.nii
Resampling: #448, 8640861_T1GD.nii.gz
Resampling: #449, 8647046_ADC.nii.gz
Resampling: #450, 8647046_FLAIR.nii.gz
Resampling: #451, 8647046_mask.nii
Resampling: #452, 8647046_T1GD.nii.gz
Resampling: #453, 8651745_ADC.nii.gz
Resampling: #454, 8651745_FLAIR.nii.gz
Resampling: #455, 8651745_mask.nii
Resampling: #456, 8651745_T1GD.nii.gz
Resampling: #457, 8662383_ADC.nii.gz
Resampling: #458, 8662383_FLAIR.nii.gz
Resampling: #459, 8662383_mask.nii
Resampling: #460, 8662383_T1GD.nii.gz
Resampling: #461, 8664725_ADC.nii.gz
Resampling: #462, 8664725_FLAIR.nii.gz
Resampling: #463, 8664725_mask.nii
Resampling: #464, 8664725_T1GD.nii.gz
Resampling: #465, 8665561_ADC.nii.gz
Resampling: #466, 8665561_FLAIR.nii.gz
Resampling: #467, 8665561_mask.nii
Resampling: #468, 8665561_T1GD.nii.gz
Resampling: #469, 8695364_ADC.nii.gz
Resampling: #470, 8695364_FLAIR

In [ ]:
# 2) N4 bias correction
total = 0
for (path, dirs, files) in os.walk(resampleDir):
    for file in files:
        total += 1
        print("N4 bias correction: #{}, {}".format(total, file))
        filePath = os.path.join(path, file)
        pt = path.split(os.path.sep)[-1]
        corr_outPath = get_outpath(correctionDir, pt, file)
        if corr_outPath is None:
            continue
        !N4BiasFieldCorrection -d 3 -i {filePath} -o {corr_outPath} -s 3 -c [50x50x50x50,0.00001] -b [150]

N4 bias correction: #1, 1_HC_0026_dti_ADC.nii.gz
1_HC_0026_dti_ADC.nii.gz already exists
N4 bias correction: #2, 1_HC_0026_dti_FA_FA.nii.gz
1_HC_0026_dti_FA_FA.nii.gz already exists
N4 bias correction: #3, T1.nii.gz
N4 bias correction: #4, 1_HC_0062_dti_ADC.nii.gz
1_HC_0062_dti_ADC.nii.gz already exists
N4 bias correction: #5, 1_HC_0062_dti_FA_FA.nii.gz
1_HC_0062_dti_FA_FA.nii.gz already exists
N4 bias correction: #6, T1.nii.gz
N4 bias correction: #7, 1_HC_0087_dti_ADC.nii.gz
1_HC_0087_dti_ADC.nii.gz already exists
N4 bias correction: #8, 1_HC_0087_dti_FA_FA.nii.gz
1_HC_0087_dti_FA_FA.nii.gz already exists
N4 bias correction: #9, T1.nii.gz
N4 bias correction: #10, 1_HC_0001_dti_ADC.nii.gz
1_HC_0001_dti_ADC.nii.gz already exists
N4 bias correction: #11, 1_HC_0001_dti_FA_FA.nii.gz
1_HC_0001_dti_FA_FA.nii.gz already exists
N4 bias correction: #12, T1.nii.gz
N4 bias correction: #13, 1_HC_0002_dti_ADC.nii.gz
1_HC_0002_dti_ADC.nii.gz already exists
N4 bias correction: #14, 1_HC_0002_dti_FA_F

In [ ]:
# 3) co-registration
total = 0
for (path, dirs, files) in os.walk(correctionDir):
    if files != []:
        total += 1
        print("coregistration: #{}, {}".format(total, files))
        pt = path.split(os.path.sep)[-1]
        
        try:
            T1C_path = glob(os.path.join(path, '*{}*'.format(T1C_label)))[0]
            #T1C_path = glob(os.path.join('/media/sc-mlsg/PYW98854602/20210104_Schizophrenia/SCH/',pt,'**','mri','T1.mgz'))[0]
        except:
            print("Target sequence not exists")
            pass
        T1C_regist_outPath = get_outpath(registerDir, pt, pt + '_{}_registered.nii'.format(T1C_label))
        if os.path.splitext(T1C_path)[-1] == '.gz':
#            !gunzip T1C_path
            T1C_regist_outPath = get_outpath(registerDir, pt, pt + '_{}_registered.nii.gz'.format(T1C_label))

        if T1C_regist_outPath is not None:
            shutil.copyfile(T1C_path, T1C_regist_outPath)
        
        for file in files:
            seq = find_seq(file)
            if seq == T1C_label:
                continue
            in_path = os.path.join(path, file)
            print("Registering {} to {}...".format(seq, T1C_label))
            outPath = get_outpath(registerDir, pt, pt + '_{}_registered.nii'.format(seq))
            if outPath is None:
                continue

            !antsRegistration -d 3 -r [{T1C_path}, {in_path}, 1] \
                              -m mattes[{T1C_path}, {in_path}, 1, 32, regular, 0.1 ] \
                              -t translation[0.1] -c [100x50x25,1.e-8,20] -s 4x2x1vox -f 6x4x2 -l 1 \
                              -m mattes[{T1C_path}, {in_path} , 1 , 32, regular, 0.1] \
                              -t rigid[0.1] -c [100x50x25,1.e-8,20]  -s 4x2x1vox  -f 3x2x1 -l 1 -o [reg, {outPath}] -v 0


In [43]:
# 4) Skull stripping using MONSTR

# we'll work in the MONSTR directory
workingDir = '/home/sc-mlsg/MONSTR'
%cd {workingDir} 
outputPath = '/home/sc-mlsg/MONSTR/output'

total = 0

for (path, dirs, files) in os.walk(registerDir):
    if files != []:
        total += 1
        print("skull stripping: #{}, {}".format(total, files))

        pt = path.split(os.path.sep)[-1]
        T1C_path = glob(os.path.join(path, '*{}*'.format(T1C_label)))[0]
        try:
            T2_path = glob(os.path.join(path, '*T2*'))[0]
        except:
            print("T2 not exists")
            T2_path = None
            
        newFileName = pt + '_brain_mask.nii.gz'
        if os.path.exists(os.path.join(brainmaskDir, newFileName)):
            print("{} already exists".format(newFileName))
            continue

        !./MONSTR.sh --t1 {T1C_path} --t2 {T2_path} --o output --atlasdir TBI_atlas --reg

        createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

        for file in os.listdir(createdDir):
            if file == 'atlas4_brainmask_ANTS.nii.gz':
                shutil.copyfile(os.path.join(createdDir, file), os.path.join(brainmaskDir, newFileName))

        shutil.rmtree(createdDir)

/home/sc-mlsg/MONSTR
skull stripping: #1, ['2034571_ADC_registered.nii', '2034571_F_registered.nii', '2034571_T1GD_registered.nii.gz']
T2 not exists
I found ANTs installation at /opt/ANTs/bin 
Default number of atlases: 4 
Number of processors is not mentioned. Using all available 6 processors. 
Atlas directory : /home/sc-mlsg/MONSTR/TBI_atlas
T1 : /media/sc-mlsg/DATA2/Perfusion_study_radiomics_preprocessed/03_Registered_corrected/2034571/2034571_T1GD_registered.nii.gz
T2 : /home/sc-mlsg/MONSTR/None
Output directory : /home/sc-mlsg/MONSTR/output 
Temporary files will be kept.
Images are assumed to be registered. T2/PD/FLAIR will not be registered again to the T1. 
Working directory : /home/sc-mlsg/MONSTR/output/2034571_T1GD_registered.nii.lRzq
cd /home/sc-mlsg/MONSTR/output/2034571_T1GD_registered.nii.lRzq
Image Exception : #63 :: No image files match: /home/sc-mlsg/MONSTR/None
terminate called after throwing an instance of 'std::runtime_error'
  what():  No image files match: /home/sc

Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas4_T1_ANTS.nii
ANTS deformable registration took 0 HRS 1 MIN 48 SEC 
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T2.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas4_T2_ANTS.nii
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_brainmask.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: NearestNeighborInterpol

Output warped image: atlas3_brainmask_ANTS.nii
AntsExample.sh t1.nii /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T1.nii fastfortesting atlas4_T1_ANTS.nii 6
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T1.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas4_T1_ANTS.nii
ANTS deformable registration took 0 HRS 1 MIN 33 SEC 
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T2.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel v

atlas1_brainmask_ANTS.nii:	 99.7% -- replaced with atlas1_brainmask_ANTS.nii.gz
atlas1_T1_ANTS.nii:	 12.8% -- replaced with atlas1_T1_ANTS.nii.gz
atlas1_T2_ANTS.nii:	 12.0% -- replaced with atlas1_T2_ANTS.nii.gz
atlas2_brainmask_ANTS.nii:	 99.7% -- replaced with atlas2_brainmask_ANTS.nii.gz
atlas2_T1_ANTS.nii:	 21.2% -- replaced with atlas2_T1_ANTS.nii.gz
atlas2_T2_ANTS.nii:	 20.4% -- replaced with atlas2_T2_ANTS.nii.gz
atlas3_brainmask_ANTS.nii:	 99.8% -- replaced with atlas3_brainmask_ANTS.nii.gz
atlas3_T1_ANTS.nii:	 25.0% -- replaced with atlas3_T1_ANTS.nii.gz
atlas3_T2_ANTS.nii:	 24.3% -- replaced with atlas3_T2_ANTS.nii.gz
atlas4_brainmask_ANTS.nii:	 99.8% -- replaced with atlas4_brainmask_ANTS.nii.gz
atlas4_T1_ANTS.nii:	 25.7% -- replaced with atlas4_T1_ANTS.nii.gz
atlas4_T2_ANTS.nii:	 25.1% -- replaced with atlas4_T2_ANTS.nii.gz
t1.nii:	 44.4% -- replaced with t1.nii.gz
Working directory was /home/sc-mlsg/MONSTR/output/2803895_T1GD_registered.nii.xxzk
MONSTR skull-stripping took

Output warped image: atlas3_brainmask_ANTS.nii
AntsExample.sh t1.nii /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T1.nii fastfortesting atlas4_T1_ANTS.nii 6
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T1.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas4_T1_ANTS.nii
ANTS deformable registration took 0 HRS 1 MIN 31 SEC 
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T2.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel v

...

Could not find library, using working directory /home/sc-mlsg/jist-lib
cli	####################################################################
cli	Initializing MIPAV/JIST Framework
Error: Heap size settings in the environment startup file do not match 
those in the Preferences file.
Memory Allocation will display so you can ensure this is correct.
fslmaths t1_MultiModalStripMask_smooth.nii t1_MultiModalStripMask_smooth.nii -odt char 
Image Exception : #63 :: No image files match: t1_MultiModalStripMask_smooth
terminate called after throwing an instance of 'std::runtime_error'
  what():  No image files match: t1_MultiModalStripMask_smooth
./MONSTR.sh: line 578: 14789 Aborted                 (core dumped) fslmaths "$subs"_MultiModalStripMask_smooth.nii "$subs"_MultiModalStripMask_smooth.nii -odt char
/home/sc-mlsg/MONSTR/image_dilate t1_MultiModalStripMask_smooth.nii t1_MultiModalStripMask_smooth.nii 1
/home/sc-mlsg/MONSTR/image_dilate: error while loading shared libraries: libmwlaunche

Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas3_T1_ANTS.nii
ANTS deformable registration took 0 HRS 1 MIN 27 SEC 
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas3_T2.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas3_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas3_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas3_T2_ANTS.nii
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas3_brainmask.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas3_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas3_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: NearestNeighborInterpol

Could not find library, using working directory /home/sc-mlsg/jist-lib
cli	####################################################################
cli	Initializing MIPAV/JIST Framework
Error: Heap size settings in the environment startup file do not match 
those in the Preferences file.
Memory Allocation will display so you can ensure this is correct.
fslmaths t1_MultiModalStripMask_smooth.nii t1_MultiModalStripMask_smooth.nii -odt char 
Image Exception : #63 :: No image files match: t1_MultiModalStripMask_smooth
terminate called after throwing an instance of 'std::runtime_error'
  what():  No image files match: t1_MultiModalStripMask_smooth
./MONSTR.sh: line 578: 15216 Aborted                 (core dumped) fslmaths "$subs"_MultiModalStripMask_smooth.nii "$subs"_MultiModalStripMask_smooth.nii -odt char
/home/sc-mlsg/MONSTR/image_dilate t1_MultiModalStripMask_smooth.nii t1_MultiModalStripMask_smooth.nii 1
/home/sc-mlsg/MONSTR/image_dilate: error while loading shared libraries: libmwlaunche

Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas3_T1_ANTS.nii
ANTS deformable registration took 0 HRS 1 MIN 24 SEC 
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas3_T2.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas3_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas3_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas3_T2_ANTS.nii
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas3_brainmask.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas3_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas3_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: NearestNeighborInterpol

Could not find library, using working directory /home/sc-mlsg/jist-lib
cli	####################################################################
cli	Initializing MIPAV/JIST Framework
Error: Heap size settings in the environment startup file do not match 
those in the Preferences file.
Memory Allocation will display so you can ensure this is correct.
fslmaths t1_MultiModalStripMask_smooth.nii t1_MultiModalStripMask_smooth.nii -odt char 
Image Exception : #63 :: No image files match: t1_MultiModalStripMask_smooth
terminate called after throwing an instance of 'std::runtime_error'
  what():  No image files match: t1_MultiModalStripMask_smooth
./MONSTR.sh: line 578: 15634 Aborted                 (core dumped) fslmaths "$subs"_MultiModalStripMask_smooth.nii "$subs"_MultiModalStripMask_smooth.nii -odt char
/home/sc-mlsg/MONSTR/image_dilate t1_MultiModalStripMask_smooth.nii t1_MultiModalStripMask_smooth.nii 1
/home/sc-mlsg/MONSTR/image_dilate: error while loading shared libraries: libmwlaunche

In [44]:
# 5) apply skull mask
total = 0
error_dict = {}

for (path, dirs, files) in os.walk(registerDir):
    if files != []:
        total += 1
        pt = path.split(os.path.sep)[-1]
        maskName = pt + '_brain_mask.nii.gz'
        mask = nib.load(os.path.join(brainmaskDir, maskName))

        for file in files:
            seq = find_seq(file)
            print("# {} Applying skull mask to {}...".format(total, file))

            # Reshape (in some cases data resolution is (x,y,z,1), reshape removes the 4th dimension
            file_nii = nib.load(os.path.join(path, file))
            file_data = file_nii.get_data()
            file_data = file_data.reshape(file_data.shape[0], file_data.shape[1], file_data.shape[2])

            outPath = get_outpath(strippedDir, pt, pt + "_{}_stripped.nii".format(seq))
            if outPath is None:
                continue

            try:
                masked_data = np.multiply(file_data, mask.get_data())
                masked_data = nib.Nifti1Image(masked_data, file_nii.affine, file_nii.header)
                nib.save(masked_data, outPath)
            except Exception as ex:
                print("Error:{}, filename:{}".format(ex.args, file))
                error_dict[pt] = ex.args
                pass

print("Finished!")
print(error_dict)

# 1 Applying skull mask to 2034571_ADC_registered.nii...


/home/sc-mlsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/sc-mlsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


# 1 Applying skull mask to 2034571_F_registered.nii...
# 1 Applying skull mask to 2034571_T1GD_registered.nii.gz...
# 2 Applying skull mask to 2803895_ADC_registered.nii...
# 2 Applying skull mask to 2803895_F_registered.nii...
# 2 Applying skull mask to 2803895_T1GD_registered.nii.gz...
# 3 Applying skull mask to 2826490_ADC_registered.nii...
# 3 Applying skull mask to 2826490_F_registered.nii...
# 3 Applying skull mask to 2826490_T1GD_registered.nii.gz...
# 4 Applying skull mask to 2917442_ADC_registered.nii...
# 4 Applying skull mask to 2917442_F_registered.nii...
# 4 Applying skull mask to 2917442_T1GD_registered.nii.gz...
# 5 Applying skull mask to 3387356_ADC_registered.nii...
# 5 Applying skull mask to 3387356_F_registered.nii...
# 5 Applying skull mask to 3387356_T1GD_registered.nii.gz...
# 6 Applying skull mask to 3441152_ADC_registered.nii...
# 6 Applying skull mask to 3441152_F_registered.nii...
# 6 Applying skull mask to 3441152_T1GD_registered.nii.gz...
# 7 Applying skull 

# 48 Applying skull mask to 7892450_F_registered.nii...
# 48 Applying skull mask to 7892450_T1GD_registered.nii.gz...
# 49 Applying skull mask to 7898233_ADC_registered.nii...
# 49 Applying skull mask to 7898233_F_registered.nii...
# 49 Applying skull mask to 7898233_T1GD_registered.nii.gz...
# 50 Applying skull mask to 7921035_ADC_registered.nii...
# 50 Applying skull mask to 7921035_F_registered.nii...
# 50 Applying skull mask to 7921035_T1GD_registered.nii.gz...
# 51 Applying skull mask to 7922825_ADC_registered.nii...
# 51 Applying skull mask to 7922825_F_registered.nii...
# 51 Applying skull mask to 7922825_T1GD_registered.nii.gz...
# 52 Applying skull mask to 7926232_ADC_registered.nii...
# 52 Applying skull mask to 7926232_F_registered.nii...
# 52 Applying skull mask to 7926232_T1GD_registered.nii.gz...
# 53 Applying skull mask to 7938576_ADC_registered.nii...
# 53 Applying skull mask to 7938576_F_registered.nii...
# 53 Applying skull mask to 7938576_T1GD_registered.nii.gz...
# 

# 95 Applying skull mask to 8494094_ADC_registered.nii...
# 95 Applying skull mask to 8494094_F_registered.nii...
# 95 Applying skull mask to 8494094_T1GD_registered.nii.gz...
# 96 Applying skull mask to 8513671_ADC_registered.nii...
# 96 Applying skull mask to 8513671_F_registered.nii...
# 96 Applying skull mask to 8513671_T1GD_registered.nii.gz...
# 97 Applying skull mask to 8518093_ADC_registered.nii...
# 97 Applying skull mask to 8518093_F_registered.nii...
# 97 Applying skull mask to 8518093_T1GD_registered.nii.gz...
# 98 Applying skull mask to 8527924_ADC_registered.nii...
# 98 Applying skull mask to 8527924_F_registered.nii...
# 98 Applying skull mask to 8527924_T1GD_registered.nii.gz...
# 99 Applying skull mask to 8529533_ADC_registered.nii...
# 99 Applying skull mask to 8529533_F_registered.nii...
# 99 Applying skull mask to 8529533_T1GD_registered.nii.gz...
# 100 Applying skull mask to 8532530_ADC_registered.nii...
# 100 Applying skull mask to 8532530_F_registered.nii...
# 10

In [5]:
# 6) z-score normalization
total = 0
for (path, dirs, files) in os.walk(strippedDir):
    for file in files:
        try:
            total += 1
            print("Z-score normalization: #{}, {}".format(total, file))
            pt = path.split(os.path.sep)[-1]
            newfilename = file.split(".")[0] + "_znormed.nii.gz"
            znorm_outPath = get_outpath(znormedDir, pt, newfilename)
            if znorm_outPath is None:
                continue
            
            seq = find_seq(file)
            if seq == "ADC":
                print("ADC, copying file...")
                img_nii = nib.load(os.path.join(path, file))
                newfilename = file.split(".")[0] + ".nii.gz"
                nib.save(img_nii, os.path.join(znormedDir, pt, newfilename))
            #    continue
            img_nii = nib.load(os.path.join(path, file))
            img_data = img_nii.get_data()
            mean, std = np.mean(img_data), np.std(img_data)
            normalized = nib.Nifti1Image((img_data - mean) / std, img_nii.affine, img_nii.header)
            nib.save(normalized, znorm_outPath)
        except Exception as e:
            print(path, file)
            print(e)

Z-score normalization: #1, 2034571_ADC_stripped.nii
ADC, copying file...


/home/sc-mlsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


Z-score normalization: #2, 2034571_FLAIR_stripped.nii
2034571_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #3, 2034571_T1GD_stripped.nii
2034571_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #4, 2803895_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #5, 2803895_FLAIR_stripped.nii
2803895_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #6, 2803895_T1GD_stripped.nii
2803895_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #7, 2826490_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #8, 2826490_FLAIR_stripped.nii
2826490_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #9, 2826490_T1GD_stripped.nii
2826490_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #10, 2917442_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #11, 2917442_FLAIR_stripped.nii
2917442_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #12, 2917442_

Z-score normalization: #89, 7716992_FLAIR_stripped.nii
7716992_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #90, 7716992_T1GD_stripped.nii
7716992_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #91, 7741219_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #92, 7741219_FLAIR_stripped.nii
7741219_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #93, 7741219_T1GD_stripped.nii
7741219_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #94, 7746738_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #95, 7746738_FLAIR_stripped.nii
7746738_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #96, 7746738_T1GD_stripped.nii
7746738_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #97, 7753977_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #98, 7753977_FLAIR_stripped.nii
7753977_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #99, 

Z-score normalization: #176, 7973273_FLAIR_stripped.nii
7973273_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #177, 7973273_T1GD_stripped.nii
7973273_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #178, 8002014_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #179, 8002014_FLAIR_stripped.nii
8002014_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #180, 8002014_T1GD_stripped.nii
8002014_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #181, 8011358_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #182, 8011358_FLAIR_stripped.nii
8011358_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #183, 8011358_T1GD_stripped.nii
8011358_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #184, 8011880_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #185, 8011880_FLAIR_stripped.nii
8011880_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalizat

Z-score normalization: #263, 8481377_FLAIR_stripped.nii
8481377_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #264, 8481377_T1GD_stripped.nii
8481377_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #265, 8483143_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #266, 8483143_FLAIR_stripped.nii
8483143_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #267, 8483143_T1GD_stripped.nii
8483143_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #268, 8484226_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #269, 8484226_FLAIR_stripped.nii
8484226_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #270, 8484226_T1GD_stripped.nii
8484226_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #271, 8485609_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #272, 8485609_FLAIR_stripped.nii
8485609_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalizat

Z-score normalization: #350, 8640861_FLAIR_stripped.nii
8640861_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #351, 8640861_T1GD_stripped.nii
8640861_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #352, 8647046_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #353, 8647046_FLAIR_stripped.nii
8647046_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #354, 8647046_T1GD_stripped.nii
8647046_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #355, 8651745_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #356, 8651745_FLAIR_stripped.nii
8651745_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalization: #357, 8651745_T1GD_stripped.nii
8651745_T1GD_stripped_znormed.nii.gz already exists
Z-score normalization: #358, 8662383_ADC_stripped.nii
ADC, copying file...
Z-score normalization: #359, 8662383_FLAIR_stripped.nii
8662383_FLAIR_stripped_znormed.nii.gz already exists
Z-score normalizat

## below are previous codes

In [ ]:
srcDir = 
    print("N4 bias correction: ", pt)
    correctionDir = os.path.join(targetDir, "correction")
    
    if not os.path.exists(correctionDir):
        os.makedirs(correctionDir)
                          
    for file in os.listdir(resampleDir):
        filePath = os.path.join(resampleDir, pt, file)
        
        if not os.path.exists(os.path.join(correctionDir, pt)):
            os.makedirs(os.path.join(correctionDir, pt))
            
        outPath = os.path.join(correctionDir, pt, file)
        #!ResampleImage 3 {filePath} {outPath} 1x1x1
        print(filePath, outPath)

In [ ]:

for (BB, T2, WB) in zip(T1_BB, T2, T1_WB):
    # patient ID number (MRN) is extracted as a variable 'mrn'
    mrn = BB.split('_')[0] 
    print((BB, T2, WB))
     print("<<< Started working on the following case: ", mrn, " >>>")
        
    # 0. Copy the current T1 and T2 files onto the working directory 
    #    (to avoid 'sudo' problem in trying to write a file on an external storage)
    pathList = []
    for file in os.listdir(patientDir):
        shutil.copyfile(os.path.join(patientDir, file), os.path.join(workingDir, file))
        pathList.append(os.path.join(workingDir, file))
    
    # 1. Resampling onto 1mm x 1mm x 1mm images 
    print("1) Resampling to 1-mm isovoxel images: ", end="")
    for file in pathList:
        !ResampleImage 3 {file} {file} 1x1x1
        print(file, end="  ")

In [ ]:
######################
##########metastasis for PYW ###########3


import os
import shutil

# we'll work in the MONSTR directory
workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 

# set paths for source and target
sourceDir = '/media/chansik/Samsung_T5/mets_nifti'
targetDir = '/media/chansik/Samsung_T5/mets_stripped'
outputPath = '/home/chansik/MONSTR/output'
##########################################################################################

alreadyDoneList = []

# file names MUST be BB.nii, WB.nii, T2.nii, FLAIR.nii, and ADC.nii

for patient in os.listdir(sourceDir):
    patientDir = os.path.join(sourceDir, patient)
    
    # check if preprocessing has already been done
    # if so, skip the patient and move onto the next patient
    if patient in os.listdir(targetDir):
        print(patient, 'has already been processed!')
        alreadyDoneList.append(patient)
        continue
    
    print("<<< Started working on the following case: ", patient, " >>>")
    
    # 0. Copy the current T1 and T2 files onto the working directory 
    #    (to avoid 'sudo' problem in trying to write a file on an external storage)
    pathList = []
    for file in os.listdir(patientDir):
        shutil.copyfile(os.path.join(patientDir, file), os.path.join(workingDir, file))
        pathList.append(os.path.join(workingDir, file))
    
    # 1. Resampling onto 1mm x 1mm x 1mm images 
    print("1) Resampling to 1-mm isovoxel images: ", end="")
    for file in pathList:
        !ResampleImage 3 {file} {file} 1x1x1
        print(file, end="  ")
    
    # 2. N4 bias correction
    # by using ANTs, using the same parameters as the default one used by MONSTR
    print("\n2) N4 bias field correction: ", end="")
    for file in pathList:
        !N4BiasFieldCorrection -d 3 -i {file} -o {file} -s 3 -c [50x50x50x50,0.00001] -b [150]
        print(file, end="  ")
  
    # 3. Co-registration
    # by using ANTs, using the same parameters as the default one used by MONSTR
    print("\n3) Co-registration with CE-T1 images as fixed, reference images: ", end="")
    for file in ["T2.nii", "WB.nii"]:
        newName = file.split(".")[0] + "_registered.nii"
        !antsRegistration -d 3 -r [BB.nii, {file}, 1] \
                          -m mattes[BB.nii, {file}, 1, 32, regular, 0.1 ] \
                          -t translation[0.1] -c [100x50x25,1.e-8,20] -s 4x2x1vox -f 6x4x2 -l 1 \
                          -m mattes[BB.nii, {file} , 1 , 32, regular, 0.1] \
                          -t rigid[0.1] -c [100x50x25,1.e-8,20]  -s 4x2x1vox  -f 3x2x1 -l 1 -o [reg, {newName}] -v 0
        print(file, end="  ")
    shutil.copyfile("BB.nii", "BB_registered.nii")
 
    # 4. MONSTR skull stripping
    t1 = "WB_registered.nii"
    t2 = "T2_registered.nii"
    !./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas --reg

    # Copy the relevant files (brain Mask and processed images)
    createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

    os.makedirs(os.path.join(targetDir, patient))
    for file in os.listdir(createdDir):
        if file == 'atlas4_brainmask_ANTS.nii.gz':
            newFileName = patient + '/brain_mask.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

    shutil.copyfile("WB_registered.nii", os.path.join(targetDir, patient, "WB_registered.nii"))
    shutil.copyfile("BB_registered.nii", os.path.join(targetDir, patient, "BB_registered.nii"))
    shutil.copyfile("T2_registered.nii", os.path.join(targetDir, patient, "T2_registered.nii"))
 
    # 5. Clean the temporarily created files and directories
    shutil.rmtree(createdDir)

In [ ]:
sourceDir = '/media/chansik/Samsung_T5/test'
targetDir = '/media/chansik/Samsung_T5/mets_stripped'
outputPath = '/home/chansik/MONSTR/output'

# Copy the relevant files (brain Mask and processed images)
createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])
patient = '1234'

os.makedirs(os.path.join(targetDir, patient))
for file in os.listdir(createdDir):
    if file == 'atlas4_brainmask_ANTS.nii.gz':
        newFileName = patient + '/brain_mask.nii.gz'
        shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))
    
shutil.copyfile("WB_registered.nii", os.path.join(targetDir, patient, "WB_registered.nii"))
shutil.copyfile("BB_registered.nii", os.path.join(targetDir, patient, "BB_registered.nii"))
shutil.copyfile("T2_registered.nii", os.path.join(targetDir, patient, "T2_registered.nii"))

# Clean the temporarily created files and directories
#os.remove(os.path.join(workingDir, t1))
#os.remove(os.path.join(workingDir, t2))
#shutil.rmtree(createdDir)


In [ ]:
t1 = "BB_registered.nii"
t2 = "T2_registered.nii"

!./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas --reg

In [ ]:
 !./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas --reg

    # 3. Copy the relevant files (brain Mask and processed T1/T2 images)
    createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

    for file in os.listdir(createdDir):
        if file == 'atlas4_brainmask_ANTS.nii.gz':
            newFileName = mrn + '_bMask.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't1.nii.gz':
            newFileName = mrn + '_T1GD_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't2.nii.gz':
            newFileName = mrn + '_T2_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

    # 4. Clean the temporarily created files and directories
    os.remove(os.path.join(workingDir, t1))
    os.remove(os.path.join(workingDir, t2))
    shutil.rmtree(createdDir)

In [ ]:
######################
##########test ###########3


import os
import shutil

# we'll work in the MONSTR directory
workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 

# set paths for source and target
sourceDir = '/media/chansik/Samsung_T5'
targetDir = '/media/chansik/SC_research_2019/GBM-Surv/Seg_results'
##########################################################################################

alreadyDoneList = []

# file names MUST be CE-T1.nii, UE-T1.nii, T2.nii, FLAIR.nii, and ADC.nii

for patient in os.listdir(sourceDir):
    patientDir = os.path.join(sourceDir, patient)
    
    # check if preprocessing has already been done
    # if so, skip the patient and move onto the next patient
    if patient in os.listdir(targetDir):
        print(patient, 'has already been processed!')
        alreadyDoneList.append(patient)
        continue
    
    print("<<< Started working on the following case: ", patient, " >>>")
    
    # 0. Copy the current T1 and T2 files onto the working directory 
    #    (to avoid 'sudo' problem in trying to write a file on an external storage)
    pathList = []
    for file in os.listdir(patientDir):
        shutil.copyfile(os.path.join(patientDir, file), os.path.join(workingDir, file))
        pathList.append(os.path.join(workingDir, file))
    
    # 1. Resampling onto 1mm x 1mm x 1mm images 
    print("1) Resampling to 1-mm isovoxel images: ", end="")
    for file in pathList:
        !ResampleImage 3 {file} {file} 1x1x1
        print(file, end="  ")
    
    # 2. N4 bias correction
    # by using ANTs, using the same parameters as the default one used by MONSTR
    print("\n2) N4 bias field correction: ", end="")
    for file in pathList:
        !N4BiasFieldCorrection -d 3 -i {file} -o {file} -s 3 -c [50x50x50x50,0.00001] -b [150]
        print(file, end="  ")
  
    # 3. Co-registration
    # by using ANTs, using the same parameters as the default one used by MONSTR
    print("\n3) Co-registration with CE-T1 images as fixed, reference images: ", end="")
    for file in ["UE-T1.nii", "T2.nii", "FLAIR.nii", "ADC.nii"]:
        newName = file.split(".")[0] + "_registered.nii"
        !antsRegistration -d 3 -r [CE-T1.nii, {file}, 1] \
                          -m mattes[CE-T1.nii, {file}, 1, 32, regular, 0.1 ] \
                          -t translation[0.1] -c [100x50x25,1.e-8,20] -s 4x2x1vox -f 6x4x2 -l 1 \
                          -m mattes[CE-T1.nii, {file} , 1 , 32, regular, 0.1] \
                          -t rigid[0.1] -c [100x50x25,1.e-8,20]  -s 4x2x1vox  -f 3x2x1 -l 1 -o [reg, {newName}] -v 0
        print(file, end="  ")
    shutil.copyfile("CE-T1.nii", "CE-T1_registered.nii")
 

In [ ]:
    # check if preprocessing has already been done
    # if so, skip the patient and move onto the next patient
    if mrn in [i.split('_', 1)[0] for i in os.listdir(targetDir)]:
        print(mrn, 'has already been processed!')
        alreadyDoneList.append(mrn)
        continue
       
    # 0. Copy the current T1 and T2 files onto the working directory 
    #    (to avoid 'sudo' problem in trying to write a file on an external storage)
    shutil.copyfile(os.path.join(sourceDir, t1), os.path.join(workingDir, t1))
    shutil.copyfile(os.path.join(sourceDir, t2), os.path.join(workingDir, t2))
    
    # 1. Resampling of CE-T1-weighted images onto 1mm x 1mm x 1mm images 
    #   (as some T1 images are 2D)
    !ResampleImage 3 {t1} {t1} 1x1x1 
    
    # 2. Skull stripping by MONSTR algorithm following 
    #          1) rigid coregistration of T2 onto T1 (ANTs)
    #          2) N4 bias correction (ANTs)
    !./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas 

    # 3. Copy the relevant files (brain Mask and processed T1/T2 images)
    createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

    for file in os.listdir(createdDir):
        if file == 'atlas4_brainmask_ANTS.nii.gz':
            newFileName = mrn + '_bMask.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't1.nii.gz':
            newFileName = mrn + '_T1GD_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't2.nii.gz':
            newFileName = mrn + '_T2_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

    # 4. Clean the temporarily created files and directories
    os.remove(os.path.join(workingDir, t1))
    os.remove(os.path.join(workingDir, t2))
    shutil.rmtree(createdDir)
    

In [ ]:
workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 
shutil.copyfile("CE-T1.nii", "CE-T1_registered.nii")

In [ ]:
workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 

!./MONSTR.sh --t1 CE-T1_registered.nii --t2 T2_registered.nii --fl FLAIR_registered.nii --o output --atlasdir TBI_atlas --reg --clean


In [ ]:
import numpy as np
import os
import nibabel as nib
#import matplotlib.pyplot as plt
#from matplotlib.image import imsave
#from PIL import Image

workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 

mask_array = nib.load("atlas4_brainmask_ANTS.nii.gz").get_fdata()
    
for file in ["UE-T1_registered.nii", "CE-T1_registered.nii", "T2_registered.nii", "FLAIR_registered.nii", "ADC_registered.nii"]:
    img_array = nib.load(file).get_fdata()
    stripped_array = img_array * mask_array
    
    newName = file.split(".")[0].split("_")[0] + "_stripped.nii"
    
    niftiFile = nib.Nifti1Image(stripped_array, np.eye(4)) 
    nib.save(niftiFile, newName)

In [ ]:
file = "UE-T1_registered.nii"
file.split(".")[0].split("_")[0]

In [ ]:
    
    
    # 2. Skull stripping by MONSTR algorithm following 
    #          1) rigid coregistration of T2 onto T1 (ANTs)
    #          2) N4 bias correction (ANTs)
    !./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas 

    # 3. Copy the relevant files (brain Mask and processed T1/T2 images)
    createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

    for file in os.listdir(createdDir):
        if file == 'atlas4_brainmask_ANTS.nii.gz':
            newFileName = mrn + '_bMask.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't1.nii.gz':
            newFileName = mrn + '_T1GD_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't2.nii.gz':
            newFileName = mrn + '_T2_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

    # 4. Clean the temporarily created files and directories
    os.remove(os.path.join(workingDir, t1))
    os.remove(os.path.join(workingDir, t2))
    shutil.rmtree(createdDir)

In [ ]:
  # 4. MONSTR skull stripping
    t1 = "WB_registered.nii"
    t2 = "T2_registered.nii"
    

    # Copy the relevant files (brain Mask and processed images)
    

    shutil.copyfile("WB_registered.nii", os.path.join(targetDir, patient, "WB_registered.nii"))
    shutil.copyfile("BB_registered.nii", os.path.join(targetDir, patient, "BB_registered.nii"))
    shutil.copyfile("T2_registered.nii", os.path.join(targetDir, patient, "T2_registered.nii"))
 
    # 5. Clean the temporarily created files and directories
    
    

## centering, cropping, and selecting slices

In [ ]:
# define a function

def center_crop(img, margin=10):
    # convert an image into an sqaure image if the dimensions of x and y are different
    if img.shape[0] != img.shape[1]:
        if img.shape[0] < img.shape[1]:
            pad_width = int((img.shape[1] - img.shape[0])//2)
            img = np.pad(img, ((pad_width, pad_width), (0,0), (0,0)), constant_values=0)

        if img.shape[0] > img.shape[1]:
            pad_width = int((img.shape[0] - img.shape[1])//2)
            img = np.pad(img, ((0,0), (pad_width, pad_width), (0,0)), constant_values=0)
        
    # calculate the non-zero margins of x and y axes.
    x_min, x_max = 1000, 0
    y_min, y_max = 1000, 0

    for z in range(img.shape[2]):
        for row in range(img.shape[0]):
            x = np.nonzero(img[row, :, z])

            if x[0].size == 0:
                continue

            if x_min > x[0].min():
                x_min = x[0].min()
            if x_max < x[0].max():
                x_max = x[0].max()

        for col in range(img.shape[1]):
            y = np.nonzero(img[:, col, z])

            if y[0].size == 0:
                continue

            if y_min > y[0].min():
                y_min = y[0].min()
            if y_max < y[0].max():
                y_max = y[0].max()

    x_width = x_max - x_min
    x_center = x_min + (x_width//2)
    y_width = y_max - y_min
    y_center = y_min + (y_width//2)

    if x_width >= y_width:
        new_width = x_width + margin
    else: 
        new_width = y_width + margin

    x_start, x_end = x_center-(new_width//2), x_center+(new_width//2)
    y_start, y_end = y_center-(new_width//2), y_center+(new_width//2)

    new_img = img[y_start:y_end, x_start:x_end, :]
    
    return new_img

In [ ]:
import pandas as pd
import numpy as np
import os
import nibabel as nib
import matplotlib.pyplot as plt
from matplotlib.image import imsave
from PIL import Image

sourceDir = '/media/chansik/SC_research_2019/GBM-Surv/ProcessedByMONSTR'
outputDir = '/home/chansik/DataAnalysis/201912-GBM-Surv/output-multiple'

range_df = pd.read_csv('/home/chansik/DataAnalysis/201912-GBM-Surv/input/range_list.csv', header=None)

target_img_size = (224, 224)

for idx, row in range_df.iterrows():
    t1_name = os.path.join(sourceDir, (str(row[0]) + "_T1GD_processed.nii.gz"))
    t2_name = os.path.join(sourceDir, (str(row[0]) + "_T2_processed.nii.gz"))
    mask_name = os.path.join(sourceDir, (str(row[0]) + "_bMask.nii.gz"))
    
    # Get Image Array (numpy array) from Nifti File
    t1_array = nib.load(t1_name).get_fdata()
    t2_array = nib.load(t2_name).get_fdata()
    mask_array = nib.load(mask_name).get_fdata()
    
    t1_img = t1_array * mask_array
    t1_img = t1_img[:, :, int(row[1]):int(row[2])]
    
    t2_img = t2_array * mask_array
    t2_img = t2_img[:, :, int(row[1]):int(row[2])]
    
    cropped_t1 = center_crop(t1_img, margin=10) # the function 'center_crop' was defined above
    cropped_t2 = center_crop(t2_img, margin=10)
    
    height = int(row[2]) - int(row[1])
    
    for slc in range(2, height, 5):
        selected_t1 = cropped_t1[:, :, slc]
        selected_t2 = cropped_t2[:, :, slc]
        
        newT1name = os.path.join(outputDir, (str(row[0]) + "_T1_" + str(slc) + ".png"))
        newT2name = os.path.join(outputDir, (str(row[0]) + "_T2_" + str(slc) + ".png"))

        imsave(newT1name, selected_t1, cmap='gray')
        imsave(newT2name, selected_t2, cmap='gray')

## resizing and stacking

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import random

baseDir = '/home/chansik/DataAnalysis/201912-GBM-Surv'
sourceDir = os.path.join(baseDir, 'output-multiple')
targetDir = os.path.join(baseDir, 'cnn_input')

df = pd.read_csv(os.path.join(baseDir, 'input/GBM-surv-list.csv'))
df = df.loc[df.death==1, ['ID', 'survival']]

short = df.loc[df.survival < df.survival.median(), 'ID'].tolist()
long = df.loc[df.survival >= df.survival.median(), 'ID'].tolist()

short_validation_list = random.choices(short, k=30)
long_validation_list = random.choices(long, k=30)

for file in os.listdir(sourceDir):
    
    # patient ID number (MRN) is extracted as a variable 'mrn'
    mrn = int(file.split('_', 1)[0])
    
    if mrn in short:
        if mrn in short_validation_list:
            shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'validation/short', file))
        else:
             shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'train/short', file))
    
    if mrn in long:
        if mrn in long_validation_list:
            shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'validation/long', file))
        else:
             shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'train/long', file))

In [ ]:
for file in os.listdir(sourceDir):
    
    # patient ID number (MRN) is extracted as a variable 'mrn'
    mrn = int(file.split('_', 1)[0])
    
    if mrn in df.ID.tolist():
        survival = df.loc[df.ID==mrn, 'survival'].values[0]
        new_fileName = str(survival) + "_" + str(mrn) + ".png"

        if mrn in validation_set:
            shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'validation', new_fileName))
        else:
            shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'train', new_fileName))
    

In [ ]:
baseDir = '/home/chansik/DataAnalysis/201912-GBM-Surv'
sourceDir = os.path.join(baseDir, 'output')
targetDir = os.path.join(baseDir, 'cnn_regression')

[p.split('_')[os.listdir(targetDir)

### divide patients into two: short and long

In [ ]:

import pandas as pd
import numpy as np
import os
import shutil

df = pd.read_csv('/home/chansik/DataAnalysis/201912-GBM-Surv/input/GBM-surv-list.csv')
df = df.loc[df.death==1, ['ID', 'survival']]

short = df.loc[df.survival < df.survival.median(), 'ID'].tolist()
long = df.loc[df.survival >= df.survival.median(), 'ID'].tolist()

sourceDir = '/home/chansik/DataAnalysis/201912-GBM-Surv/output'
shortDir = '/home/chansik/DataAnalysis/201912-GBM-Surv/cnn_input/short'
longDir = '/home/chansik/DataAnalysis/201912-GBM-Surv/cnn_input/long'

for file in os.listdir(sourceDir):
    
    # patient ID number (MRN) is extracted as a variable 'mrn'
    mrn = int(file.split('_', 1)[0])

    if mrn in short:
        shutil.copyfile(os.path.join(sourceDir, file), os.path.join(shortDir, file))
    if mrn in long:
        shutil.copyfile(os.path.join(sourceDir, file), os.path.join(longDir, file))

## convert gray to RGB

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import tensorflow as tf
from matplotlib.image import imsave

baseDir = '/home/chansik/DataAnalysis/201912-GBM-Surv'
print(os.path.join(baseDir, 'output'))
for file in os.listdir(os.path.join(baseDir, 'output')):
    final_img_name = os.path.join(baseDir, 'gray-to-rgb', file)
    converted = tf.image.grayscale_to_rgb(file, name=None)
    imsave(final_img_name, converted)
    


In [ ]:
from PIL import Image
resized_image = Image.fromarray(new_img[:,:,20]).resize((224, 224))

nifti_file = nib.Nifti1Image(new_img, np.eye(4))
nib.save(nifti_file, 'test.nii') # Save

In [ ]:
plt.imshow(np.array(resized_image))

In [ ]:
plt.imshow(img[:, :, 29])

In [ ]:
print(img.shape)
new = np.pad(img, pad_width = ((1,1), (0,0), (0,0)), constant_values=0)
print(new.shape)

In [ ]:
import pandas as pd
import numpy as np
import os
import nibabel as nib
import matplotlib.pyplot as plt
from matplotlib.image import imsave
from PIL import Image

image_array = nib.load("251020_T1GD_processed.nii.gz").get_fdata()
mask_array = nib.load("251020_bMask.nii.gz").get_fdata()
img = image_array * mask_array
img = img[:, :, 53:113]
 